# Investigation of Wiki revisions API

We will start by investigating the revisions of Captain Marvel, and then we will generalize to all of the superheroes provided by the course.

## Data setup

In [1]:
import pandas as pd

df_marvel = pd.read_csv("../files/marvel.csv", index_col=0, encoding='utf8')
df_marvel = df_marvel[df_marvel["WikiLink"].notna()]
df_dc = pd.read_csv("../files/dc.csv", index_col=0, encoding='utf8')
df_dc = df_dc[df_dc["WikiLink"].notna()]

df_marvel['universe'] = 'Marvel'
df_dc['universe'] = 'DC'

df = pd.concat([df_marvel, df_dc], ignore_index=True, axis=0)

In [72]:
df.head()

,CharacterName,WikiLink,universe
0,Abomination,Abomination (character),Marvel
1,Absorbing Man,Absorbing Man,Marvel
2,Achebe,Achebe (comics),Marvel
3,Agent,Agent (comics),Marvel
4,Agent X,Agent X (Marvel Comics),Marvel


## API call

In [21]:
cap_wikilink = "Señor Muerte / Señor Suerte" #df[df.CharacterName=="Captain America"].WikiLink.item()
#cap_name = 
cap_wikilink

'Señor Muerte / Señor Suerte'

In [22]:
from urllib import request
import os
for wikilink in cap_wikilink.split('|'):
    baseurl = 'https://en.wikipedia.org/w/api.php?'
    action = 'action=query'
    title = f'titles={wikilink.replace(" ", "_")}'
    content = 'prop=revisions&rvprop=content|ids|userid|timestamp&rvlimit=500'
    dataformat = 'format=json'

    query = f'{baseurl}{action}&{title}&{content}&{dataformat}'

    wikiresponse = request.urlopen(query)
    wikidata = wikiresponse.read()
    wikitext = wikidata.decode('utf-8')

    import json
    wikijson = json.loads(wikitext)
    revisions = list(wikijson['query']['pages'].values())[0]['revisions']
    print(len(revisions))
    #for revision in revisions:
    #    path = 'superheroes/' + cap_name + '/' + wikilink
    #    isExist = os.path.exists(path)
    #    if not isExist:
    #        os.makedirs(path)
        
    #    with open(path + '/' + f"{str(revision['revid'])+'-'+str(revision['timestamp'])}.txt", "w") as text_file:
    #        text_file.write(str(revision['*']))
        


UnicodeEncodeError: 'ascii' codec can't encode character '\xf1' in position 37: ordinal not in range(128)

In [82]:
list(wikijson['query']['pages'].values())[0]['revisions'][1]

{'revid': 1111577664,
 'parentid': 1111279616,
 'userid': 40451009,
 'timestamp': '2022-09-21T18:54:57Z',
 'contentformat': 'text/x-wiki',
 'contentmodel': 'wikitext',
 '*': '{{Short description|Fictional character}}\n{{Redirect|I can do this all day|the song Save the City|Hawkeye (2021 TV series)#Music}}\n{{Distinguish|Captain America (Ultimate Marvel character)}}\n{{Use American English|date=August 2020}}\n{{Use mdy dates|date=July 2021}}\n{{Infobox character\n| color       = #AF181F\n| name        = Steve Rogers\n| franchise   = [[Marvel Cinematic Universe]]\n| image       = Chris Evans as Steve Rogers Captain America.jpg\n| caption     = [[Chris Evans (actor)|Chris Evans]] as Steve Rogers / Captain America in \'\'[[Avengers: Age of Ultron]]\'\' (2015)\n| first       = \'\'[[Captain America: The First Avenger|Captain America:<br /> The First Avenger]]\'\' (2011)\n| last        = \'\'[[Avengers: Endgame]]\'\' (2019)\n| based_on    = {{Based on|[[Captain America]]|[[Joe Simon]]|[[Jack

In [45]:
wikijson['query']['pages']['7729']['revisions']

[{'revid': 1112270091,
  'parentid': 1110769895,
  'userid': 7903804,
  'contentformat': 'text/x-wiki',
  'contentmodel': 'wikitext',
  '*': '{{short description|Comic book superhero}}\n{{Other uses}}\n{{redirect|Steve Rogers}}\n{{redirect|Steve Rogers (Marvel Comics)|the film character|Steve Rogers (Marvel Cinematic Universe)}}\n{{pp-semi-indef}}\n{{use mdy dates|date=July 2021}}\n{{Infobox comics character <!--Wikipedia:WikiProject Comics-->\n| character_name  = Steve Rogers<br>{{small|\'\'\'Captain America\'\'\'}}\n| image     = Captain America (Steve Rogers).png\n| converted = \n| caption   = Captain America, as he appeared on the cover of \'\'[[Tales of Suspense]]\'\' #74 (February 1966)<br />Art by [[Jack Kirby]]\n| alt       = Captain America bursting through a page of newspaper\n| alter_ego = Steven Rogers\n| publisher = [[Marvel Comics]]\n| debut     = \'\'[[#Golden Age|Captain America Comics]]\'\' #1 (March 1941)\n| creators  = {{plainlist|\n*[[Joe Simon]]\n*[[Jack Kirby]]}}\

Notice that a character can have multiple sides, which is should be taken into account, but let us save that problem for a rainy day.

In [103]:
import requests

for wikilink in cap_wikilink.split('|'):
    BASE_URL = "http://en.wikipedia.org/w/api.php"
    parameters = { 'action': 'query',
               'format': 'json',
               'titles': wikilink.replace(" ", "_"),
               'prop': 'revisions',
               'rvprop': 'ids|userid|content|timestamp',
               'rvlimit': 'max'}

    wp_call = requests.get(BASE_URL, params=parameters)
    response = wp_call.json()

    print(response)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'continue': {'rvcontinue': '20181106081421|867526527', 'continue': '||'}, 'warnings': {'main': {'*': 'Subscribe to the mediawiki-api-announce mailing list at <https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/> for notice of API deprecations and breaking changes. Use [[Special:ApiFeatureUsage]] to see usage of deprecated features by your application.'}, 'revisions': {'*': 'Because "rvslots" was not specified, a legacy format has been used for the output. This format is deprecated, and in the future the new format will always be used.'}}, 'query': {'normalized': [{'from': 'Captain_America_(Ultimate_Marvel_character)', 'to': 'Captain America (Ultimate Marvel character)'}], 'pages': {'1774501': {'pageid': 1774501, 'ns': 0, 'title': 'Captain America (Ultimate Marvel character)', 'revisions': [{'revid': 1110556293, 'parentid': 1110496205, 'anon': '', 'userid': 0, 'timestamp': '2022-09-16T05:19:48Z', 'contentformat': 'text/x-wiki', 'contentmodel': 'wikit

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [104]:
len(list(response['query']['pages'].values())[0]['revisions'])

50

In [19]:
df_comics[df_comics.CharacterName=="Señor Muerte / Señor Suerte"]

,CharacterName,WikiLink,universe
1349,Señor Muerte / Señor Suerte,Señor Muerte / Señor Suerte,Marvel


In [20]:
import requests

BASE_URL = "http://en.wikipedia.org/w/api.php"
TITLE = 'Señor Muerte / Señor Suerte'
for link in TITLE.split('|'):
    parameters = { 'action': 'query',
               'format': 'json',
               'continue': '',
               'titles': link,
               'prop': 'revisions',
               'rvprop': 'ids|timestamp',
               'rvlimit': 'max'}

    wp_call = requests.get(BASE_URL, params=parameters)
    response = wp_call.json()

    total_revisions = 0

    revid_timestamp = dict()
        
    while True:
        wp_call = requests.get(BASE_URL, params=parameters)
        response = wp_call.json()
        
        for page_id in response['query']['pages']:
            total_revisions += len(response['query']['pages'][page_id]['revisions'])
            response['query']['pages'][page_id]['revisions']
            
            for revision in response['query']['pages'][page_id]['revisions']:
                revid_timestamp[revision['timestamp']] = revision['revid']
                
        if 'continue' in response:
            parameters['continue'] = response['continue']['continue']
            parameters['rvcontinue'] = response['continue']['rvcontinue']

        else:
            break
    
    print(len(list(revid_timestamp.keys())))
    print()
    print(parameters['titles'], total_revisions)

44

Señor Muerte / Señor Suerte 44


In [134]:
revid_timestamp

{'2022-09-25T16:16:00Z': 1112289538,
 '2022-09-21T18:54:57Z': 1111577664,
 '2022-09-20T05:09:45Z': 1111279616,
 '2022-09-19T16:05:12Z': 1111158599,
 '2022-09-19T16:04:59Z': 1111158565,
 '2022-09-18T20:37:07Z': 1111008021,
 '2022-09-18T19:51:28Z': 1111000990,
 '2022-09-18T19:48:57Z': 1111000690,
 '2022-09-16T06:17:39Z': 1110562240,
 '2022-09-14T22:14:33Z': 1110335158,
 '2022-09-13T22:41:08Z': 1110155982,
 '2022-09-13T15:11:46Z': 1110096345,
 '2022-09-09T17:17:52Z': 1109408187,
 '2022-09-08T03:53:11Z': 1109128989,
 '2022-09-08T03:22:44Z': 1109126365,
 '2022-09-08T03:07:21Z': 1109125070,
 '2022-09-08T03:02:54Z': 1109124647,
 '2022-09-06T22:37:33Z': 1108908256,
 '2022-08-26T01:07:59Z': 1106704607,
 '2022-08-18T22:41:33Z': 1105171753,
 '2022-08-18T22:40:20Z': 1105171627,
 '2022-08-18T08:37:08Z': 1105055611,
 '2022-08-18T08:04:26Z': 1105052303,
 '2022-08-17T20:23:17Z': 1104967116,
 '2022-08-02T03:31:00Z': 1101845820,
 '2022-08-02T00:54:48Z': 1101825586,
 '2022-07-27T21:25:25Z': 1100816983,
 

In [170]:
import datetime

#Converting timestamp to datetime object
new_format = "%Y-%m-%d %H:%M:%S"
converted_keys = {datetime.datetime.strptime(time_stamp, "%Y-%m-%dT%H:%M:%SZ").strftime(new_format):revid for time_stamp, revid in revid_timestamp.items()}
converted_keys

{'2022-09-25 16:16:00': 1112289538,
 '2022-09-21 18:54:57': 1111577664,
 '2022-09-20 05:09:45': 1111279616,
 '2022-09-19 16:05:12': 1111158599,
 '2022-09-19 16:04:59': 1111158565,
 '2022-09-18 20:37:07': 1111008021,
 '2022-09-18 19:51:28': 1111000990,
 '2022-09-18 19:48:57': 1111000690,
 '2022-09-16 06:17:39': 1110562240,
 '2022-09-14 22:14:33': 1110335158,
 '2022-09-13 22:41:08': 1110155982,
 '2022-09-13 15:11:46': 1110096345,
 '2022-09-09 17:17:52': 1109408187,
 '2022-09-08 03:53:11': 1109128989,
 '2022-09-08 03:22:44': 1109126365,
 '2022-09-08 03:07:21': 1109125070,
 '2022-09-08 03:02:54': 1109124647,
 '2022-09-06 22:37:33': 1108908256,
 '2022-08-26 01:07:59': 1106704607,
 '2022-08-18 22:41:33': 1105171753,
 '2022-08-18 22:40:20': 1105171627,
 '2022-08-18 08:37:08': 1105055611,
 '2022-08-18 08:04:26': 1105052303,
 '2022-08-17 20:23:17': 1104967116,
 '2022-08-02 03:31:00': 1101845820,
 '2022-08-02 00:54:48': 1101825586,
 '2022-07-27 21:25:25': 1100816983,
 '2022-07-27 13:18:51': 1100

In [168]:
list(converted_keys.keys())[-1]

'2018-01-13:02:12:54'

In [160]:
import pandas as pd
import numpy as np

In [190]:

df = pd.DataFrame.from_dict({'timestamp':list(converted_keys.keys()),'rev_id':list(converted_keys.values())})          # Load a dataframe with your file
df['timestamp'] = pd.to_datetime(df['timestamp'],  format='%Y-%m-%d %H:%M:%S')
dfg = df.groupby(pd.Grouper(key='timestamp', freq='Q'))  # group by month / alternatively use MS for Month Start / referencing the previously created object

# Finally, find the max date in each month
selected_timestamps = dfg.agg({'timestamp': np.max}).timestamp.astype(str).tolist()
converted_keys[selected_timestamps[0]]

833487268

# Putting it all together

In [26]:
import pandas as pd

df_marvel = pd.read_csv("../files/marvel.csv", index_col=0, encoding='utf8')
df_marvel = df_marvel[df_marvel["WikiLink"].notna()]
df_dc = pd.read_csv("../files/dc.csv", index_col=0, encoding='utf8')
df_dc = df_dc[df_dc["WikiLink"].notna()]

df_marvel['universe'] = 'Marvel'
df_dc['universe'] = 'DC'

df_comics = pd.concat([df_marvel, df_dc], ignore_index=True, axis=0)

In [ ]:
import requests
import datetime
import json
from tqdm import tqdm
import numpy as np
from urllib import request
from urllib.parse import quote
import os

BASE_URL = "http://en.wikipedia.org/w/api.php"

for idx, row in tqdm(df_comics.iterrows()):
    wikilink = row["WikiLink"]
    character_name = row["CharacterName"]
    for link in wikilink.split('|'):
        parameters = { 'action': 'query',
                   'format': 'json',
                   'continue': '',
                   'titles': link,
                   'prop': 'revisions',
                   'rvprop': 'ids|timestamp',
                   'rvlimit': 'max'}

        wp_call = requests.get(BASE_URL, params=parameters)
        response = wp_call.json()

        total_revisions = 0

        revid_timestamp = dict()

        while True:
            wp_call = requests.get(BASE_URL, params=parameters)
            response = wp_call.json()
        

            for page_id in response['query']['pages']:
                
                if 'revisions' in response['query']['pages'][page_id].keys():
                
                    total_revisions += len(response['query']['pages'][page_id]['revisions'])
                    response['query']['pages'][page_id]['revisions']

                    for revision in response['query']['pages'][page_id]['revisions']:
                        revid_timestamp[revision['timestamp']] = revision['revid']

            if 'continue' in response:
                parameters['continue'] = response['continue']['continue']
                parameters['rvcontinue'] = response['continue']['rvcontinue']

            else:
                break

        #Selecting timestamps to query
        new_format = "%Y-%m-%d %H:%M:%S"
        converted_keys = {datetime.datetime.strptime(time_stamp, "%Y-%m-%dT%H:%M:%SZ").strftime(new_format):revid for time_stamp, revid in revid_timestamp.items()}
        converted_keys
        df = pd.DataFrame.from_dict({'timestamp':list(converted_keys.keys()),'rev_id':list(converted_keys.values())})
        df['timestamp'] = pd.to_datetime(df['timestamp'],  format='%Y-%m-%d %H:%M:%S')
        dfg = df.groupby(pd.Grouper(key='timestamp', freq='Y'))  # Q: group by quarter

        # Finally, find the max date in each quater
        selected_timestamps = dfg.agg({'timestamp': np.max}).timestamp.astype(str).tolist()
        #converted_keys[selected_timestamps[0]]

        for timestamp in selected_timestamps:
            if timestamp == "NaT":
                continue
            baseurl = 'https://en.wikipedia.org/w/api.php?'
            action = 'action=query'
            #title = f'titles={link.replace(" ", "_")}'
            content = 'prop=revisions&rvprop=content|ids|userid|timestamp'
            dataformat = 'format=json'
            revids = f'revids={int(converted_keys[timestamp])}'

            #query = f'{baseurl}{action}&{revids}&{title}&{content}&{dataformat}'
            query = f'{baseurl}{action}&{revids}&{content}&{dataformat}'

            wikiresponse = request.urlopen(query)
            wikidata = wikiresponse.read()
            wikitext = wikidata.decode('utf-8')

            wikijson = json.loads(wikitext)
            revision = list(wikijson['query']['pages'].values())[0]['revisions'][0]
            #print(len(list(wikijson['query']['pages'].values())[0]['revisions']))
            if '*' in list(revision.keys()):
                path = 'superheroes/' + character_name + '/' + link
                isExist = os.path.exists(path)
                if not isExist:
                    os.makedirs(path)

                with open(path + '/' + f"{str(converted_keys[timestamp])+'-'+str(timestamp)}.txt", "w") as text_file:
                    text_file.write(str(revision['*']))


119it [16:16,  6.90s/it]

In [31]:
response['query']['pages'][page_id]

{'ns': 0, 'title': 'L1=Master Man', 'missing': ''}